In [1]:
from itertools import groupby
import tensorflow as tf
import random
import torch
import numpy as np
####### READING THE FASTA CHAIN DATA FROM THE FILE ############

def fasta_iter(name):
    fh = open(name)

    faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))

    for header in faiter:
        # drop the ">"
        headerStr = header.__next__()[1:].strip()

        seq = "".join(s.strip() for s in faiter.__next__())

        yield (headerStr, seq)

data=fasta_iter("Ecoli_prom.fa.txt")
training_data=[]

######## CONVERTING DNA SEQUENCE INTO ONE HOT ENCODING ##########

for chain in data:
    headerStr, seq = chain
    #print(seq)
    temp=[]
    for i in seq:
        if(i=='A'):
            temp.append([1,0,0,0])
        if(i=='T'):
            temp.append([0,1,0,0])
        if(i=='G'):
            temp.append([0,0,1,0])
        if(i=='C'):
            temp.append([0,0,0,1])
    
    training_data.append(temp)
       
    
    
data=fasta_iter("Ecoli_non_prom.fa.txt")
for chain in data:
    headerStr, seq = chain
    temp=[]
    for i in seq:
        if(i=='A'):
            temp.append([1,0,0,0])
        if(i=='T'):
            temp.append([0,1,0,0])
        if(i=='G'):
            temp.append([0,0,1,0])
        if(i=='C'):
            temp.append([0,0,0,1])
        
    training_data.append(temp)

#training_data

In [2]:
########### CREATING LABELS FOR THE OUTPUT ###########
labels=[]
for i in range(839):
    labels.append(1)
for i in range(3000):
    labels.append(0)

#### SHUFFLE THE DATA ########    

comb=list(zip(training_data,labels))
import time
random.seed(time.time())

random.shuffle(comb)
training_data,labels=zip(*comb)


print(np.shape(training_data))
print(np.shape(labels))

training_data=np.reshape(training_data,(3839,81*4))

print(np.shape(training_data))


(3839, 81, 4)
(3839,)
(3839, 324)


In [3]:
####### SPLITTING DATA INTO TRAIN,VALIDATION AND TEST ############
X_train=training_data[:3000]
Y_train=labels[:3000]

X_valid=training_data[3000:3400]
Y_valid=labels[3000:3400]



X_test=training_data[3400:]
Y_test=labels[3400:]

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV


/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.

In [5]:

# parameters1 = {
#               'min_samples_leaf': [1,2,3], 
#               'n_estimators': [30,400]
#              }
# rfc = RandomForestClassifier(random_state=42)
# RF_model = GridSearchCV(estimator=rfc,param_grid=parameters1,cv=5)
# RF_model.fit(X_train,Y_train)




In [6]:
parameters = {'bootstrap': True,
              'min_samples_leaf': 3,
              'n_estimators': 200,
              'min_samples_split': 4,
              'max_features': None,
              'max_depth': 6,
              'warm_start':True
             }

RF_model = RandomForestClassifier(**parameters)
RF_model.fit(X_train, Y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [7]:
RF_predictions = RF_model.predict(X_train)
score = accuracy_score(Y_train ,RF_predictions)
print(score*100)

90.96666666666667


In [8]:
RF_predictions = RF_model.predict(X_valid)
score = accuracy_score(Y_valid ,RF_predictions)
print(score*100)

89.5


In [9]:
RF_predictions = RF_model.predict(X_test)
score = accuracy_score(Y_test ,RF_predictions)
print(score*100)

88.15489749430525


##### 